## (Technical) Jupyter Notebook Standardizing Identifier Order In Adjacent Columns in hu.MAP2-provided CSV

This parallels [the notebook 'Jupyter Notebook Standardizing Identifier Order In Adjacent Columns in hu.MAP2-provided CSV'](https://nbviewer.org/github/fomightez/humap3-binder/blob/main/additional_nbs/standardizing_initial_data/Standardizing_identifier_order_in_humap3-provided_csv.ipynb) to investigate if author-provided hu.MAP 2.0 data displays the same issues and fix if it does.   
This will set things up so it will be convenient to compare author-provided hu.MAP 2.0 data to author-provided hu.MAP 3.0 data.


------

**Details**, i.e., the long story:

Authors provided a CSV about the hu.MAP 2.0 complex.  Does it have issues like hu.MAP 3.0 data? If so, address it like I did there.

This will set things up so it will be convenient to compare author-provided hu.MAP 2.0 data to author-provided hu.MAP 3.0 data.

-------

### Preparation


##### Get the complexes with confidence scores

Because the author-provided source didn't work for the hu.MAP 3.0 data, I expected `curl -OL "http://humap2.proteincomplexes.org/static/downloads/humap2/humap2_complexes_20200809.txt"` to work on my local machine, yet fail on MyBinder because the involved port may be blocked on MyBinder for getting it from the original resource. Because of that expectation, I made a copy at https://gist.githubusercontent.com/fomightez/af3edda957e4d71acbaa30192e74e9af/raw/108a8c3fb3374a74ef3ca5d772a9dfe96e996c93/humap2_complexes_20200809.txt where MyBinder would have access. However, the curl of the original source works!! 
(Keeping a note about my copy now but using original sourc

In [1]:
!curl -OL "http://humap2.proteincomplexes.org/static/downloads/humap2/humap2_complexes_20200809.txt"
# If that fails, uncomment & try the next line which will be guaranteed to work with MyBinder:
#!curl -OL https://gist.githubusercontent.com/fomightez/af3edda957e4d71acbaa30192e74e9af/raw/108a8c3fb3374a74ef3ca5d772a9dfe96e996c93/humap2_complexes_20200809.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  500k  100  500k    0     0  1472k      0 --:--:-- --:--:-- --:--:-- 1475k


##### Put the data on the complexes into Pandas dataframe

(I'm using uv here just because I want to learn about it. I could have run the code in the script right in this notebook, and skipped the pickling and read pickle steps.)

Get the script to use with `uv` to read in the raw data and make a dataframe.

In [2]:
!curl -OL https://raw.githubusercontent.com/fomightez/structurework/refs/heads/master/humap3-utilities/complexes_rawCSV_to_df.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1007  100  1007    0     0   3882      0 --:--:-- --:--:-- --:--:--  3888


In [3]:
!uv run complexes_rawCSV_to_df.py humap2_complexes_20200809.txt
import pandas as pd
rd_df = pd.read_pickle('raw_complexes_pickled_df.pkl')
rd_df

Reading inline script metadata from `complexes_rawCSV_to_df.py`
Installed 10 packages in 199ms                                       


,HuMAP2_ID,Confidence,Uniprot_ACCs,genenames
0,HuMAP2_00000,3,Q9BQS8 O95900,FYCO1 TRUB2
1,HuMAP2_00001,4,P08133 Q15797 Q99426 Q9H4M9 P68402 Q15102,ANXA6 SMAD1 TBCB EHD1 PAFAH1B2 PAFAH1B3
2,HuMAP2_00002,5,Q93062 Q9NZC3 Q9UF11 Q15038 Q6ZRY4 A1KXE4 O432...,RBPMS GDE1 PLEKHB1 DAZAP2 RBPMS2 FAM168B RBFOX...
3,HuMAP2_00003,5,Q15836 Q16563 Q29983 Q8WUM9 O14974 Q9Y5Y0 Q149...,VAMP3 SYPL1 MICA SLC20A1 PPP1R12A FLVCR1 DRAP1...
4,HuMAP2_00004,4,Q8WV99 Q9NQT8 Q9H672 P20774 Q49A92,ZFAND2B KIF13B ASB7 OGN C8orf34
...,...,...,...,...
6960,HuMAP2_07014,4,Q9HC97 Q92871 Q6S8J3 P13727 P31152,GPR35 PMM1 POTEE PRG2 MAPK4
6961,HuMAP2_07015,4,Q9H5L6 Q8N5N7 Q96E29 O75127 Q9NPE2 Q96I51,THAP9 MRPL50 MTERF3 PTCD1 NGRN RCC1L
6962,HuMAP2_07016,5,Q99697 Q8NE31 P17509 P31274 Q2T9J0 Q8TAC2 P529...,PITX2 FAM13C HOXB6 HOXC9 TYSND1 JOSD2 NKX2-5 D...
6963,HuMAP2_07017,2,Q53GT1 Q96GP6 P49448,KLHL22 SCARF2 GLUD2


Initial idea to make in-order was to collect the identifiers in the Uniprot_ACCs & genenames columns by exploding the columns **together** and then making a list that is reduced to just the unique set. Then that would serve as a basis making a lookup table to relate the contents of the two columns.    
But that doesn't work for both columns together as I had discovered, & cover next the reasoning, and so the code is left with what works and that is just 'exploding' one column:

In [5]:
# Can use Pandas `explode()`-related code I worked out in relation to using `rd_df` data already (see `notebooks/Working_with_hu.MAP3_data_with_Python_in_Jupyter_Basics.ipynb`)
# to make the step of flattening the Uniprot_ACCs
intermed_df = rd_df.copy()
intermed_df['Uniprot_ACCs'] = intermed_df['Uniprot_ACCs'].str.split()
#intermed_df['genenames'] = intermed_df['genenames'].str.split() # cannot use explode with both columns because out of 6965 rows, 204 rows don't have same number of indentifiers in the two columns (?!?! Do some identifiers occur twice in one or other?)
# Now use explode to create a new row for each element in both columns
expanded_df = intermed_df.explode(['Uniprot_ACCs']).copy()
# Reset the index 
expanded_df = expanded_df.reset_index(drop=True)
len(expanded_df)

30573

That didn't work as I had thought for both columns, read on for as to why and how fixed so ultimately exploding those two columns together will work (since the code I made made elsewhere and used was assuming 'exploding' the columns would work well)....

#### Detour to see why mismatching number of identifiers in the two columns, `Uniprot_ACCs` & `genenames`, in 204 rows

Noted something curious while drafting the code for the above cell..

Found couldn't use `explode()` with both columns of the entire 'raw data' dataframe because out of 15326 rows, 279 rows (see next cell showing this) don't have same number of indentifiers in the two columns (?!?! Do some identifiers occur twice in one or other?)

In [6]:
intermed_df = rd_df.copy()
intermed_df['Uniprot_ACCs'] = intermed_df['Uniprot_ACCs'].str.split()
intermed_df['genenames'] = intermed_df['genenames'].str.split()
#intermed_df.head()
num = 0
for row in intermed_df.itertuples():
    if len(row.Uniprot_ACCs) != len(row.genenames):
        num+=1
import rich
rich.print(f"Total rows in the CSV: [bold black]{len(intermed_df)}[/bold black]")
rich.print(f"Total Rows where the number of identifiers in the\ntwo adjacent columns aren't the same: [bold black]{num}[/bold black]")

Total rows in the CSV: 6965

Total Rows where the number of identifiers in the
two adjacent columns aren't the same: 204

Show some examples that can be explored:

In [7]:
# Filter rows where list lengths differ
i_df = rd_df.copy()
i_df['Uniprot_ACCs'] = i_df['Uniprot_ACCs'].str.split()
i_df['genenames'] = i_df['genenames'].str.split()
mismtchd = i_df[i_df['Uniprot_ACCs'].apply(len) != i_df['genenames'].apply(len)]

# Randomly select 10 rows from the filtered DataFrame
sample_rows = mismtchd.sample(n=10, random_state=3) #use state to assure sampling is same when re-run
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
    display(sample_rows)

,HuMAP2_ID,Confidence,Uniprot_ACCs,genenames
6492,HuMAP2_06542,5,"[P52756, Q12872, Q96I25, O15042, Q8IWX8, P98175]","[RBM5, SFSWAP, ZRSR2P1, RBM17, U2SURP, CHERP, RBM10]"
6360,HuMAP2_06409,5,"[Q14954, P43631, Q14953, Q8TAX7]","[KIR2DS1, KIR2DS2, KIR2DS5, KIR3DP1, MUC7]"
4077,HuMAP2_04110,5,"[Q9Y620, Q96EW2, P43355, Q8IWZ3, Q07617, O15480, O15481, Q8N7X4, Q96A19]","[RAD54B, HSPBAP1, MAGEA1, ANKHD1, SPAG1, MAGEB3, FBXL18, MAGEB4, MAGEB6, CCDC102A]"
5662,HuMAP2_05705,1,[Q8N6C8],"[LILRA3, LILRB2]"
5385,HuMAP2_05424,5,"[Q86TB9, Q6PJG9]","[PATL1, LRFN4, IGHM]"
2516,HuMAP2_02536,2,[P01023],"[A2M, PZP]"
5885,HuMAP2_05931,4,"[Q00978, Q9BU19, P42224, P52630]","[RPS26P11, IRF9, ZNF692, STAT1, STAT2]"
5216,HuMAP2_05255,3,"[Q8N6C8, Q8NHL6]","[LILRA3, LILRB2, LILRB1]"
928,HuMAP2_00932,5,"[P22392, O00746, Q13232, P22392, Q9H4I3]","[NME2P1, NME2, NME4, NME3, NME2, TRABD]"
2518,HuMAP2_02538,4,"[Q86UD1, Q15545]","[OAF, TAF7, TRAC]"


This is a nice sampling of examples to examine.

I can clearly see several don't have equal numbers in each column.
- Example from row #2516 `[P01023]	[A2M, PZP]`. Second column has two genenames, while first has one identifier.
- Example from row #5216: `[Q8N6C8, Q8NHL6]	[LILRA3, LILRB2, LILRB1]`. Two identifiers in first columns and three in second.
- Example from row #928: `[P22392, O00746, Q13232, P22392, Q9H4I3]	[NME2P1, NME2, NME4, NME3, NME2, TRABD]` Five identifiers in first column and six in second column.
  
In [the notebook 'Jupyter Notebook Standardizing Identifier Order In Adjacent Columns in hu.MAP2-provided CSV'](https://nbviewer.org/github/fomightez/humap3-binder/blob/main/additional_nbs/standardizing_initial_data/Standardizing_identifier_order_in_humap3-provided_csv.ipynb) I had worked through similar examples in greater detail to gain insight into what was going on, and at the same time, start to understand maybe how to address the issues.  
**Here though**, I am going to proceed for now and assume the issues are similar to seen and addressed there, or at least I covered most of the possibilities, since this is a smaller subset and the same authors processed the data it seems.  
**UPDATE**: later, mostly around the time I had generated a preliminary output I called `DRAFThumap2_complexes_20200809InOrderMatched.csv` by **initially**(pretty much[I had iterated adding some hu.MAP 2.0-specific handling a bit before saving that]) running the hu.MAP 3.0-handling code on the hu.MAP 2.0, I found beyond the possibilities already covered in hu.MAP 3.0, examples involving synonyms use and where authors used gene names no longer represented in UniProt and had to add special handling for those identifiers in accounting for things to set up for keeping balanced & checking for balance in the two columns. Plus, the handling for those with multiple names seems different in the hu.MAP 2.0 vs. hu.MAP 3.0 data. (Inconsistent handling scheme highlights that without this standardization it would be hard to compare the hu.MAP 2.0 vs. hu.MAP 3.0 data, which I am still trying to find if they did. I can see mention of increases but about cases where they lost members of complexes?)

Here are some examples of the types of things I mention in the update:  
Then further looking into those cases found things like following in author-provided `humap2_complexes_20200809.txt`:
```text
HuMAP2_06154,4,  Q86YV6 Q9H7B4   Q9H9G7,HSP90AA5P HSP90AB2P MYLK4 SMYD3 HSP90AB3P HSP90AB4P AGO3
```

`HSP90AB4P` corresponds to UniProt accension `Q58FF6`, but that isn't present there. Instead weird gap seen between `Q9H7B4   Q9H9G7`?
`HSP90AB2P` corresponds to UniProt accension `Q58FF8`, but that isn't present there. Instead weird gap seen between `Q9H7B4   Q9H9G7`?
`HSP90AB3P` corresponds to UniProt accension `Q58FF7`, but that isn't present there. Instead weird gap seen between `Q9H7B4   Q9H9G7`?
`HSP90AA5P` corresponds to UniProt accension `Q58FG0`, but that isn't present there. Instead weird gap seen between `Q9H7B4   Q9H9G7`?
Why so unbalanced?

```text
HuMAP2_00547,5, P58876 P16104 P23527 Q96T23 Q6ZRQ5 Q8IUE6,H3-2 HIST1H2BD H2AFX HIST1H2BO RSF1 MMS22L HIST2H2AB
```

I can account for all six identifiers in the `Uniprot_ACCs` column, as long as I add in using synonymys, but there is no corresponding identifier for `H3-2` that seems to correspond to `Q5TEC6`, if I had to guess. Why unbalanced?

```text
HuMAP2_01550,4,Q8NB90  Q9BW66 Q9BVQ7 Q9NX04,SPATA5 RIPK4 CINP SPATA5L1 C1orf109
```

Note the extended gap between: `Q8NB90  Q9BW66`. What is that about and why unbalanced number of items?    
When I look, I see `Q8NB90` corresponds to `AFG2A` and a synonym is `SPATA5`. So that is accounted for.  
Note the gap would be here and that above and below account for the ones flanking `RIPK4`.
When I look, I see `Q9BW66` correspnds to `CINP`. So that is accounted for.  
When I look, I see `Q9BVQ7` corresponds  to `AFG2B` and a synonym is `SPATA5L1`. So that is accounted for.  
When I look, I see `Q9NX04` corresponds to `AIRIM` and a synonym is `
C1orf109`. So that is accounted for.  
Turns out as the gap hinted, `RIPK4` seems to be the one missing a corresponding entry in the `Uniprot_ACCs` column. It seems it should be `P57078`.

```text
HuMAP2_00006,4,P07199 Q96C28 O43296  Q5CZA5,CENPB ZNF707 ZNF264 ZNF678 ZNF805
```
Four on one side and five on the other. ZNF678 probably corresponds to Q5SXM1.  
Note the extended gap between: `O43296  Q5CZA5`. What is that about and why unbalanced number of items?  
`P07199` corresponds to `CENPB`.  
`Q96C28` corresponds to `ZNF707`.  
`O43296` corresponds to `ZNF264`.
Note the gap would be here and that above and below account for the ones flanking `ZNF678`.  
`Q5CZA5` corresponds to `ZNF805`.
Turns out as the gap hinted, `ZNF678` seems to be the one missing a corresponding entry in the `Uniprot_ACCs` column. It seems it should be `Q5SXM1`.


```text
HuMAP2_00011,5, O00339 Q99435 Q9BXX0 Q92832 Q3ZCT1,C4orf48 MATN2 NELL2 EMILIN2 NELL1 ZNF260
```

Based on seeing a gap there and `C4orf48` coinciding and columns not balancing, I'm guessing `C4orf48` should have a matching entry of `Q5BLP8`.  

```text
HuMAP2_03893,2,Q9H8T0 Q96ED9 Q05DH4 Q86VS8 Q8N612 Q9UJC3,AKTIP HOOK2 FAM160A1 HOOK3 FAM160A2 HOOK1
```

```text
HuMAP2_04870,5,P04439 ,HLA-A LOC100507703
```

One in one column and two in the other? There seems to be a gap there after `P04439`: `P04439 ,`?

```text
HuMAP2_04875,4,Q5JXC2  P09917,MIIP RUNDC1 ALOX5
```

Two in one column and three in the other?  There seems to be a gap there between `Q5JXC2` & `P09917`: `Q5JXC2  P09917,`?

```text
HuMAP2_06623,4, P30511 Q9NPC4 Q8NDV1 O15466,KLRA1P HLA-F A4GALT ST6GALNAC3 ST8SIA5
```

Four in one column and five in the other?  There seems to be a gap there before `P30511`?

```text
HuMAP2_04079,5,O75069 P62805  Q8WUP2 P68431 Q8N806 P68431 Q13111,TMCC2 H4C1 H3-2 FBLIM1 H3C1 UBR7 H3C1 CHAF1A
```

`P68431` occurs twice?

For `P68431` just use single genename, unlike complex one involving semi-colons in hu.MAP 3.0 data, and so in the code below I had to adjust the special handling to do the accounting correctly to not add additional identifiers n the `Uniprot_ACCs` column when trying to balance.  

```text
HuMAP2_04092,4,Q8WXA9 O95232 P0DN76 Q13523 Q05519 Q8IZL8 Q99590 Q14498 Q7L014 P26368 P0DN76,SREK1 LUC7L3 U2AF1L5 PRPF4B SRSF11 PELP1 SCAF11 RBM39 DDX46 U2AF2 U2AF1L5
HuMAP2_04092,4,Q8WXA9 O95232 P0DN76 Q13523 Q05519 Q8IZL8 Q99590 Q14498 Q7L014 P26368 P0DN76,SREK1 LUC7L3 U2AF1L5 PRPF4B SRSF11 PELP1 SCAF11 RBM39 DDX46 U2AF2 U2AF1L5
```
`P0DN76` occurs twice?

```text
HuMAP2_02212,2,P59665 P59665,DEFA1 DEFA1
```

`P59665` occurs twice and that is the only protein in this 'complex'?

```text
HuMAP2_01368,5,Q9Y3F4 P08621 O75534 Q9P2G9 Q7Z422 Q9H840 Q9BRS8 O14893 Q9H9B4 Q9NWZ8 O43353 Q8WXD5 P57678 Q16637 Q16637 Q9H3H3 Q9UHI6 Q9P2E3 P83369 Q8TEQ6,STRAP SNRNP70 CSDE1 KLHL8 SZRD1 GEMIN7 LARP6 GEMIN2 SFXN1 GEMIN8 RIPK2 GEMIN6 GEMIN4 SMN1 SMN1 C11orf68 DDX20 ZNFX1 LSM11 GEMIN5
```

`SMN1` occurs twice?

I would imagine these duplicates, if plentiful, could throw off comparison of hu.MAP 2.0 vs hu.MAP 3.0 data especially if use count of associated complex members anywhere in assessment. And so I checked how many, see `Check_for_duplicates_in_author_provided_hu.MAP2.0_csv.ipynb`, and I'll be sure to fix on the standardized version so won't be an issue in downstream efforts. (Or maybe already fixed by way I do things?) 

**END OF UPDATE WHERE I HAD ORIGINALLY PROCEEDED AND FOUND LOTS OF ADDITIONAL HANDLING NEEDED AND INCONSISTENCIES WITH HOW HANDLED IN hu.MAP 3.0 data and came back to this point in notebook and noted**

### Make lookup table for relating identifiers in the two columns, column Uniprot_ACCs and column genenames

I already made a large lookup table for relating identifiers in the effort for [the notebook 'Jupyter Notebook Standardizing Identifier Order In Adjacent Columns in hu.MAP2-provided CSV'](https://nbviewer.org/github/fomightez/humap3-binder/blob/main/additional_nbs/standardizing_initial_data/Standardizing_identifier_order_in_humap3-provided_csv.ipynb).   
Here there are only 6965 rows vs. the 15326 there. Plus, there's only 30573 identifiers in the '`Uniprot_ACCs`' column while there was 75531 there. So most of the identifiers probably handled in this lookup table.

There I used the identifiers in the the '`Uniprot_ACCs`' column to look them up the UniProt Knowledgebase using the package Unipressed (see my [Unipressed-binder repo](https://github.com/fomightez/Unipressed-binder) for more on working with this package).  
So I can take what I generated from there and see if it already covers what I need and go from there.

In [8]:
# first make a list of all the identifiers in the 'Uniprot_ACCs' column
ids_in_Uniprot_ACCs = list(set(expanded_df['Uniprot_ACCs'].to_list())) # the set will limit to unique set, eliminating ones that appear more than once
len(ids_in_Uniprot_ACCs)

9963

There's 9963 unique identifiers among all the identifiers in the 'Uniprot_ACCs' column in this hu.MAP 2.0 data. There's 13769 unique identifiers among all the identifiers in the 'Uniprot_ACCs' column in the hu.MAP 3.0 data.
I can read what I had from there in here and then see what isn't covered.

In [12]:
lu_file_needed = "look_up_dict_all_13769_Uniprot_ACCs.pkl"
import os
if not os.path.isfile(lu_file_needed):
    !curl -OL "https://github.com/fomightez/humap3-binder/raw/refs/heads/main/additional_nbs/standardizing_initial_data/{lu_file_needed}"
import time
time.sleep(1.0)
import pickle
with open("look_up_dict_all_13769_Uniprot_ACCs.pkl", "rb") as f:
        h3_lookup_dict = pickle.load(f)
len(list(h3_lookup_dict.keys()))

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  233k  100  233k    0     0   589k      0 --:--:-- --:--:-- --:--:--  589k


13769

Now that I have the 13769 identifiers in the hu.MAP 3.0 data read in, I can see if there are any in the hu.MAP 2.0 data not already there by running set analysis on the two.

In [13]:
h2 = set(ids_in_Uniprot_ACCs)
h3 = set(h3_lookup_dict.keys())
missing_in_h3 = h2 - h3
if missing_in_h3:
  print(f"There's {len(missing_in_h3)} identifiers in h2 are not present in h3.")
else:
  print("All items in h2 are present in h3")

There's 419 identifiers in h2 are not present in h3.


While 419 is not insignificant, as it is way more than I'd want to handle by hand, it won't take a long time to look those up compared to the hours it took to collect all the matching gene names for the 13769.
So here I want to expand on the lookup table stored in `"look_up_dict_all_13769_Uniprot_ACCs.pkl"` and use that expanded lookup table for the effort here.

In [9]:
'''ONLY RUN THIS WHEN NEED TO COLLECT LOOKUP INFORMATION!!!
# This first cell sets things up for querying UniProt; THIS COMES FROM https://nbviewer.org/github/fomightez/humap3-binder/blob/main/additional_nbs/standardizing_initial_data/Standardizing_identifier_order_in_humap3-provided_csv.ipynb
from unipressed import UniprotkbClient
import time
import requests
import pickle
import os

def process_uniprot_chunk(id_list):
    """
    Process a chunk of UniProt IDs and return a dictionary of ID to gene name mappings.
    """
    chunk_dict = {}
    for uniprot_id in id_list:
        try:
            uniprot_record = UniprotkbClient.fetch_one(uniprot_id)
            #lookup_dict[uniprot_id] = uniprot_record['genes'][0]['geneName']['value'] #worked for when one 'gene' in the list returned by `uniprot_record['genes']`
            # HOWEVER...
            # some IDs like `Q96LI6` and `Q9BQ83` give more than one gene for `uniprot_record['genes']` and the CSV from the hu.MAP3 people was combining 
            # those to things like `HSFY1; HSFY2` and `SLX1A; SLX1B`, respectively. To do accurate accounting and keep close to that (or at least keep the option to keep close to that), I want to recapitulate that, too.
            if 'genes' in uniprot_record:
                chunk_dict[uniprot_id] = '; '.join([x['geneName']['value'] for x in uniprot_record['genes']])
            else:
                if uniprot_id == 'B3KT37': # special case I looked into
                    chunk_dict[uniprot_id] = 'SPECIALin_UniProt_and_VETELKLIC_part_similar_to_YWHAE_but_no_official_gene'
                else:
                    chunk_dict[uniprot_id] = 'SPECIALin_UniProt_but_no_gene'
        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 400:
                chunk_dict[uniprot_id] = 'not_known'
                print(f"UniProt ID '{uniprot_id}' not found. Marked as 'not_known' in lookup_dict.\n(Although for known case, '645345', this will be fixed later.)")
            else:
                raise e
        time.sleep(1.12)
    return chunk_dict

def process_all_ids_in_chunks(id_list, chunk_size=1000, output_dir='chunk_pickles', resume=True):
    """
    Process all IDs in chunks and save each chunk as a pickle file.
    Supports resuming from the last completed chunk.
    
    Args:
        id_list (list): List of all UniProt IDs to process
        chunk_size (int): Size of each chunk
        output_dir (str): Directory to save pickle files
        resume (bool): If True, skip already processed chunks
    """
    os.makedirs(output_dir, exist_ok=True)
    
    total_chunks = (len(id_list) + chunk_size - 1) // chunk_size
    
    # Find the last completed chunk if resuming
    existing_chunks = []
    if resume:
        existing_chunks = [f for f in os.listdir(output_dir) if f.startswith('chunk_') and f.endswith('.pkl')]
        existing_chunks.sort()  # Sort to find the highest number
    
    # Determine starting chunk
    start_chunk = 0
    if existing_chunks:
        last_chunk = existing_chunks[-1]
        start_chunk = int(last_chunk.split('_')[1].split('.')[0])  # Extract number from 'chunk_003.pickle'
        print(f"Resuming from chunk {start_chunk + 1}")
    
    # Process remaining chunks
    for i in range(start_chunk * chunk_size, len(id_list), chunk_size):
        chunk_num = i // chunk_size + 1
        chunk = id_list[i:i + chunk_size]
        
        pickle_filename = os.path.join(output_dir, f'chunk_{chunk_num:03d}.pkl')
        
        # Skip if chunk already exists and we're resuming
        if resume and os.path.exists(pickle_filename):
            print(f"Skipping existing chunk {chunk_num}/{total_chunks}")
            continue
            
        print(f"Processing chunk {chunk_num}/{total_chunks} ({len(chunk)} IDs)")
        chunk_result = process_uniprot_chunk(chunk)
        
        with open(pickle_filename, 'wb') as f:
            pickle.dump(chunk_result, f)
        
        print(f"Saved {len(chunk_result)} entries to {pickle_filename}")

def combine_pickle_chunks(pickle_dir='chunk_pickles'):
    """
    Combine all pickle files in the directory into a single dictionary.
    
    Args:
        pickle_dir (str): Directory containing the pickle files
    
    Returns:
        dict: Combined dictionary of all chunks
    """
    combined_dict = {}
    pickle_files = sorted([f for f in os.listdir(pickle_dir) if f.endswith('.pkl')])
    
    for pickle_file in pickle_files:
        with open(os.path.join(pickle_dir, pickle_file), 'rb') as f:
            chunk_dict = pickle.load(f)
            combined_dict.update(chunk_dict)
        print(f"Loaded {pickle_file}, combined dictionary now has {len(combined_dict)} entries")
    
    return combined_dict
''';

In [10]:
'''ONLY RUN THIS WHEN NEED TO COLLECT LOOKUP INFORMATION!!!
process_all_ids_in_chunks(list(missing_in_h3), chunk_size=500)
''';

(I looked into it some, I cannot tell where `missing_in_h3` is introducing `nan` that isn't even a string?!??! I don't think it will cause much issue so letting it be there for now and moving on.)

In [42]:
'''ONLY RUN THIS WHEN NEED TO COLLECT LOOKUP INFORMATION!!!
missing_lookup_dict = combine_pickle_chunks()
''';

Loaded chunk_001.pkl, combined dictionary now has 419 entries


Now I need to add that `missing_lookup_dict` to `h3_lookup_dict`. The next cell should do that.

In [44]:
'''ONLY RUN THIS WHEN NEED TO COLLECT LOOKUP INFORMATION!!!
merged_lookup_dict = h3_lookup_dict.copy()
merged_lookup_dict.update(missing_lookup_dict)
''';

In [46]:
''''''ONLY RUN THIS WHEN NEED TO COLLECT LOOKUP INFORMATION!!!
len(list(merged_lookup_dict.keys()))
''';

14188

As it doesn't take up too much space (list 242K), store the lookup table in pickled form for having in case need for reference later. Or for re-running downstream steps without needing to make the lookup table again. 

In [51]:
'''ONLY RUN THIS WHEN NEED TO COLLECT LOOKUP INFORMATION!!!
import pickle
with open("look_up_dict_for_h2nh3_all_14188_Uniprot_ACCs.pkl", "wb") as f:
        pickle.dump(merged_lookup_dict, f)
''';

Because it didn't take up too much space, I stored the lookup table in pickled form for having in case need for reference later. Or fo re-running without needing to make again. So optional code below to handle those optons. (Otherwise, in the way the code is in the cell, the cell runs fine without doing anything in present form.)

In [ ]:
# if need to read in the GIANT lookup dictonary
'''
import pickle
with open("look_up_dict_for_h2nh3_all_14188_Uniprot_ACCs.pkl", "rb") as f:
        merged_lookup_dict = pickle.load(f)
len(list(merged_lookup_dict.keys()))
'''

# if need to save the GIANT lookup dictionary again for some reason
'''
import pickle
with open("look_up_dict_for_h2nh3_all_14188_Uniprot_ACCs.pkl", "wb") as f:
        pickle.dump(merged_lookup_dict, f)
'''

### Now use lookup table to fix and balance the two columns, `'Uniprot_ACCs'` & `'genenames'`

This is a step towards making a better CSV that adheres close to the author-provided one but follows better data management practices to make it easier to use Pandas `explode()` the way I had been when I though the two columns were in-order matched and I had not realized the extent of special cases buried in the data.

Also will do some accounting along the way so I can have more numbers to relate what was going on in original CSV. One aspect of this will be to assess how many special cases there are and make it easy to find them more easily.

In [14]:
import pickle
with open("look_up_dict_for_h2nh3_all_14188_Uniprot_ACCs.pkl", "rb") as f:
        merged_lookup_dict = pickle.load(f)
len(list(merged_lookup_dict.keys()))

14188

Found **later** needed some additional ones added that have gene names in the `genename` column but no corresponding identifier in the `Uniprot_ACCs` column , like so:

```text
HuMAP2_06154,4,  Q86YV6 Q9H7B4   Q9H9G7,HSP90AA5P HSP90AB2P MYLK4 SMYD3 HSP90AB3P HSP90AB4P AGO3
HuMAP2_00547,5, P58876 P16104 P23527 Q96T23 Q6ZRQ5 Q8IUE6,H3-2 HIST1H2BD H2AFX HIST1H2BO RSF1 MMS22L HIST2H2AB
HuMAP2_01550,4,Q8NB90  Q9BW66 Q9BVQ7 Q9NX04,SPATA5 RIPK4 CINP SPATA5L1 C1orf109
```

Next cell handles the several cases of that:

In [16]:
'''
Add handling for following:
`HSP90AB2P` corresponds to UniProt accension `Q58FF8`, but that isn't present there. Instead weird gap seen between `Q9H7B4   Q9H9G7`?
`HSP90AB4P` corresponds to UniProt accension `Q58FF6`, but that isn't present there. Instead weird gap seen between `Q9H7B4   Q9H9G7`?
`HSP90AB3P` corresponds to UniProt accension `Q58FF7`, but that isn't present there. Instead weird gap seen between `Q9H7B4   Q9H9G7`?
`HSP90AA5P` corresponds to UniProt accension `Q58FG0`, but that isn't present there. Instead weird gap seen between `Q9H7B4   Q9H9G7`?
`H3-2` that seems to correspond to `Q5TEC6`
`RIPK4` seems to be the one missing a corresponding entry in the `Uniprot_ACCs` column. It seems it should be `P57078`.
`ZNF678` that seems to correspond to `Q5SXM1`
`C4orf48` should have a matching entry of `Q5BLP8`. 
WASH6P Q9NQA3 
IGHG4 P01861
LINC01587 Q99440
FAM90A26 D6RGX4
FAM90A5P A8MXJ8
MSL3P1 P0C860
PI4KAP1 Q8N8J0
ZRSR2P1 Q15695
DENND10P1 Q6NSW5
LILRB2 Q8N423
ALDH3B2 P48448
FAM153CP A0AAQ5BID4
FAM153B P0C7A2
FRG1BP SPECIAL_HGNC15792 (couldn't find one but this is HGNC entry accession)
TUBB7P SPECIAL_HGNC12413 (couldn't find one but this is HGNC entry accession)
CHCHD2P9 Q5T1J5
OFCC1 Q8IZS5
SNRPGP15 A8MWD9
IGHA1 P01876
NME2P1 O60361
DSCR4 P56555
PABPC4L P0CB38
IGLC7 A0M8Q6
SSX6P Q7RTT6
GPATCH4 Q5T3I0
CEP170P1 Q96L14
FRMD8P1 Q9BZ68
CROCCP2 Q86T23
OR4K3 Q96R72
FBXL18 Q96ME1
HMGB1P1 B2RPK0
FBLL1 A6NHQ2
WASH3P C4AMC7
ABHD18 Q0P651
PZP P20742
IGLC3 P0DOY3
HECTD4 Q9Y4D8
SOWAHA Q2M3V2
SIGLEC16 A6NMB1
TEX15 Q9BXT5
PPP1R15B Q5SWA1
PRSS46P E5RG02
PLEKHA8P1 O95397
SAC3D1 A6NKF1
KIR3DP1 A0A0G2JN01
MRPL45 Q9BRJ2
NEDD8-MDP1 E9PL57
HSPA7 P48741
ARMCX4 Q5H9R4
IGHM P01871
PIPSL A2A3N6
APOBEC3D Q96AK3
POTEKP Q9BYX7
IGKC P01834
CIRBP-AS1 Q8TBR5
ZSCAN12 O43309
APOA4 P06727
IGHG1 P01857
CTSL3P Q5NE16
ZNF724 A8MTY0
PRKY O43930
NPIPB7 O75200
DHRS4L2 Q6PKH6
HLA-H P01893
IGHA2 P01877
PRR5-ARHGAP8 B1AHC3
EP400P1 Q6ZTU2
PATJ Q8NI35
LILRB3 O75022
TRAC P01848
POM121C A8CG34
IGLC2 P0DOY2
LILRA6 Q6PI73
IGHG3 P01860
IGHG2 P01859
RUNDC1 Q96C34
RPS26P11 Q5JNZ5
HSP90AA4P Q58FG1
LINC01667 SPECIAL_HGNC52455 (couldn't find one but this is HGNC entry accession)
LOC100507703  SPECIAL_HLA-A-related   (https://www.ncbi.nlm.nih.gov/gene/100507703 says LOC100507703 replaced with Gene ID: 3105 but that is HLA-A that is already list in complex)
KLRA1P O75889
MRRFP1 SPECIAL_HGNC35238 (couldn't find one but this is HGNC entry accession)
INTS4P2 SPECIAL_HGNC22351 (couldn't find one but this is HGNC entry accession)
C9orf106 Q8NAJ2
GGT2 P36268
GK3P Q14409
RPSAP58 A0A8I5KQE6
USP41 SPECIAL_HGNC20070 (couldn't find one but this is HGNC entry accession)
'''
special_genename_lookup_dict = {}
special_genename_lookup_dict['HSP90AB4P'] = 'Q58FF6'
special_genename_lookup_dict['HSP90AB3P'] = 'Q58FF7'
special_genename_lookup_dict['HSP90AB2P'] = 'Q58FF8'
special_genename_lookup_dict['HSP90AA5P'] = 'Q58FG0'
special_genename_lookup_dict['H3-2'] = 'Q5TEC6'
special_genename_lookup_dict['RIPK4'] = 'P57078'
special_genename_lookup_dict['ZNF678'] = 'Q5SXM1'
special_genename_lookup_dict['C4orf48'] = 'Q5BLP8'
special_genename_lookup_dict['WASH6P'] = 'Q9NQA3'
special_genename_lookup_dict['IGHG4'] = 'P01861'
special_genename_lookup_dict['LINC01587'] = 'Q99440'
special_genename_lookup_dict['FAM90A26'] = 'D6RGX4'
special_genename_lookup_dict['FAM90A5P'] = 'A8MXJ8'
special_genename_lookup_dict['MSL3P1'] = 'P0C860'
special_genename_lookup_dict['PI4KAP1'] = 'Q8N8J0'
special_genename_lookup_dict['ZRSR2P1'] = 'Q15695'
special_genename_lookup_dict['DENND10P1'] = 'Q6NSW5'
special_genename_lookup_dict['LILRB2'] = 'Q8N423'
special_genename_lookup_dict['ALDH3B2'] = 'P48448'
special_genename_lookup_dict['FAM153CP'] = 'A0AAQ5BID4'
special_genename_lookup_dict['FAM153B'] = 'P0C7A2'
special_genename_lookup_dict['FRG1BP'] = 'SPECIAL_HGNC15792' #couldn't find one but this is HGNC entry accession
special_genename_lookup_dict['TUBB7P'] = 'SPECIAL_HGNC12413' #couldn't find one but this is HGNC entry accession
special_genename_lookup_dict['CHCHD2P9'] = 'Q5T1J5'
special_genename_lookup_dict['OFCC1'] = 'Q8IZS5'
special_genename_lookup_dict['SNRPGP15'] = 'A8MWD9'
special_genename_lookup_dict['IGHA1'] = 'P01876'
special_genename_lookup_dict['NME2P1'] = 'O60361'
special_genename_lookup_dict['DSCR4'] = 'P56555'
special_genename_lookup_dict['PABPC4L'] = 'P0CB38'
special_genename_lookup_dict['IGLC7'] = 'A0M8Q6'
special_genename_lookup_dict['SSX6P'] = 'Q7RTT6'
special_genename_lookup_dict['GPATCH4'] = 'Q5T3I0'
special_genename_lookup_dict['CEP170P1'] = 'Q96L14'
special_genename_lookup_dict['FRMD8P1'] = 'Q9BZ68'
special_genename_lookup_dict['CROCCP2'] = 'Q86T23'
special_genename_lookup_dict['OR4K3'] = 'Q96R72'
special_genename_lookup_dict['FBXL18'] = 'Q96ME1'
special_genename_lookup_dict['HMGB1P1'] = 'B2RPK0'
special_genename_lookup_dict['FBLL1'] = 'A6NHQ2'
special_genename_lookup_dict['WASH3P'] = 'C4AMC7'
special_genename_lookup_dict['ABHD18'] = 'Q0P651'
special_genename_lookup_dict['PZP'] = 'P20742'
special_genename_lookup_dict['IGLC3'] = 'P0DOY3'
special_genename_lookup_dict['HECTD4'] = 'Q9Y4D8'
special_genename_lookup_dict['SOWAHA'] = 'Q2M3V2'
special_genename_lookup_dict['SIGLEC16'] = 'A6NMB1'
special_genename_lookup_dict['TEX15'] = 'Q9BXT5'
special_genename_lookup_dict['PPP1R15B'] = 'Q5SWA1'
special_genename_lookup_dict['PRSS46P'] = 'E5RG02'
special_genename_lookup_dict['PLEKHA8P1'] = 'O95397'
special_genename_lookup_dict['SAC3D1'] = 'A6NKF1'
special_genename_lookup_dict['KIR3DP1'] = 'A0A0G2JN01'
special_genename_lookup_dict['MRPL45'] = 'Q9BRJ2'
special_genename_lookup_dict['NEDD8-MDP1'] = 'E9PL57'
special_genename_lookup_dict['HSPA7'] = 'P48741'
special_genename_lookup_dict['ARMCX4'] = 'Q5H9R4'
special_genename_lookup_dict['IGHM'] = 'P01871'
special_genename_lookup_dict['PIPSL'] = 'A2A3N6'
special_genename_lookup_dict['APOBEC3D'] = 'Q96AK3'
special_genename_lookup_dict['POTEKP'] = 'Q9BYX7'
special_genename_lookup_dict['IGKC'] = 'P01834'
special_genename_lookup_dict['CIRBP-AS1'] = 'Q8TBR5'
special_genename_lookup_dict['ZSCAN12'] = 'O43309'
special_genename_lookup_dict['APOA4'] = 'P06727'
special_genename_lookup_dict['IGHG1'] = 'P01857'
special_genename_lookup_dict['CTSL3P'] = 'Q5NE16'
special_genename_lookup_dict['ZNF724'] = 'A8MTY0'
special_genename_lookup_dict['PRKY'] = 'O43930'
special_genename_lookup_dict['NPIPB7'] = 'O75200'
special_genename_lookup_dict['DHRS4L2'] = 'Q6PKH6'
special_genename_lookup_dict['HLA-H'] = 'P01893'
special_genename_lookup_dict['IGHA2'] = 'P01877'
special_genename_lookup_dict['PRR5-ARHGAP8'] = 'B1AHC3'
special_genename_lookup_dict['EP400P1'] = 'Q6ZTU2'
special_genename_lookup_dict['PATJ'] = 'Q8NI35'
special_genename_lookup_dict['LILRB3'] = 'O75022'
special_genename_lookup_dict['TRAC'] = 'P01848'
special_genename_lookup_dict['POM121C'] = 'A8CG34'
special_genename_lookup_dict['IGLC2'] = 'P0DOY2'
special_genename_lookup_dict['LILRA6'] = 'Q6PI73'
special_genename_lookup_dict['IGHG3'] = 'P01860'
special_genename_lookup_dict['IGHG2'] = 'P01859'
special_genename_lookup_dict['RUNDC1'] = 'Q96C34'
special_genename_lookup_dict['RPS26P11'] = 'Q5JNZ5'
special_genename_lookup_dict['HSP90AA4P'] = 'Q58FG1'
special_genename_lookup_dict['LINC01667'] = 'SPECIAL_HGNC52455' #couldn't find one but this is HGNC entry accession
special_genename_lookup_dict['LOC100507703'] = 'SPECIAL_HLA-A-related' #couldn't find one LOC100507703, seems HLA-A-related 
special_genename_lookup_dict['KLRA1P'] = 'O75889' 
special_genename_lookup_dict['MRRFP1'] = 'SPECIAL_HGNC35238' #couldn't find one but this is HGNC entry accession
special_genename_lookup_dict['INTS4P2'] = 'SPECIAL_HGNC22351' #couldn't find one but this is HGNC entry accession
special_genename_lookup_dict['C9orf106'] = 'Q8NAJ2' 
special_genename_lookup_dict['GGT2'] = 'P36268' 
special_genename_lookup_dict['GK3P'] = 'Q14409'
special_genename_lookup_dict['RPSAP58'] = 'A0A8I5KQE6'
special_genename_lookup_dict['USP41'] = 'SPECIAL_HGNC20070' #couldn't find one but this is HGNC entry accession

Now ready to get started fixing!

In [20]:
# now using the `merged_lookup_dict` go through `rd_df` dataframe row by row and 
# replace the column Uniprot_ACCs and genenames with in-order matched, balanced
# versions and do accounting on them to track things along the way.
# The tracking dataframe that will result will be separate from the main one 
# because I need that to be good to save and replace the author-provided `hu.MAP3.0_complexes_wConfidenceScores_total15326_wGenenames_20240922.csv`
lookup_dict = merged_lookup_dict
fixed_df = rd_df.copy()
tracker_results_df = rd_df.copy()
class Tracker:
    def __init__(self):
        self.tracking_list = []

def make_ordered_fix_and_collect_info_about_balance(row, tracker):
    # for each row iterate on the identfiers in the column Uniprot_ACCs and 
    # account for the corresponding genename value in the genenames column. 
    # THIS WILL PRODUCED SAME NUMBER IN EACH COLUMN - so they are BALANCED in 
    # new version.
    # Do some tracking for accounting, too:
    # What is left? If anything is left then `genenames_had_unaccounted_for`
    # Also note if matching originally. Or if even balanced originally. (Note
    # that 'balanced originally' will take into account what I know about some 
    # Uniprot accensions corresponding to two genenames to produce the ones like 
    # `SLX1A; SLX1B` or `HSFY1; HSFY2` that have the semi-colon between the two 
    # related genemaes for the single Uniprot_ACC.
    # Or if one of the semi-colons is involved, etc.
    new_Uniprot_ACCs_list = []
    new_genenames_list = []
    original_ACCs_string = row['Uniprot_ACCs']
    original_ACCs_as_list = row['Uniprot_ACCs'].split()
    original_ACCs_as_list = list(set(original_ACCs_as_list)) #do this to remove duplicates, see above & `Check_for_duplicates_in_author_provided_hu.MAP2.0_csv.ipynb` showing quite a few occurences of duplicates in rows of raw data
    original_genenames_string = row['genenames']
    original_genenames_as_list = row['genenames'].split()
    # if original `genenames` has semi-colons, fix list based on what I had seen
    # in analysis earlier in this notebook, so what looks like two separated by
    # a `; ` are one without a space at all in the list. This is necessary to 
    # check the number of items balanced in each column later for 
    # 'balanced_originally' assessnent
    if ';' in original_genenames_string:
        # then fix the list
        adjusted_original_genenames_as_list = []
        i = 0
        while i < len(original_genenames_as_list):
            if original_genenames_as_list[i].endswith(';'):
                new_string = original_genenames_as_list[i] + original_genenames_as_list[i+1]
                adjusted_original_genenames_as_list.append(new_string)
                i += 2  # Skip the next element because already merged it into one before it to make one genename text string corresponding to the ID for a specific single Uniprot_ACC; for checking for balance
            else:
                adjusted_original_genenames_as_list.append(original_genenames_as_list[i])
                i += 1
        original_genenames_as_list = adjusted_original_genenames_as_list # now set the list to the fixed one
    what_remains = original_genenames_string # will be used to see if any genenames content not accounted for by content in Uniprot_ACCs column
    matching_originally = False # set this and only change if established below
    balanced_originally = False # set this and only change if established below
    involves_semicolon = False # set this and only change if established below
    SPECIAL_involved = False # set this and only change if established below
    genenames_had_unaccounted_for = False # set this and only change if established below
    what_genenames_had_unaccounted_for = "NOTHING" # set this and only change if established below

    # Sort the original_ACCs_as_list based on the length of their corresponding genenames
    # in lookup_dict (longest first)
    sorted_original_ACCs = sorted(
        original_ACCs_as_list,
        key=lambda x: len(lookup_dict[x]),
        reverse=True
    )
    # Now process in order of longest genename to shortest so for related genes like `TBL1X` and `TBL1XR1`, I don't end up removing `TBL1X` from `TBL1XR1` and just leave `R1` when making `what_remains`.
    for original_ACC in sorted_original_ACCs:
        if original_ACC == '645345': # handle this special case I noted where Uniprot_ACCs doesn't match a UniProt accessions; and this way if authors ever fix this won't happen and won't affect anything
            new_Uniprot_ACCs_list.append('Q5T1J5')
        else:
            new_Uniprot_ACCs_list.append(original_ACC) # TYPICAL FOR ALL EXCEPT ONE!
        matched_genename = lookup_dict[original_ACC]
        new_genenames_list.append(matched_genename.replace(" ","")) # the replace will remove the space after the semi-colon for any that have that; maybe I could have done it when I made the lookup table but I had decided at the time to keep the option to adhere closer to the orginal author-provided CSV representation but now realize when 'exploding' the contents of the columns to have one identifier pairing per row, the space after the semi-colon will be an issue in the way I did it based on my earlier assessment of the author-provided CSV. On the plus side of leaving it in, when I remove the matching text from the text in the genenames column to see what may remain unaccounted for based on the IDs in the Uniprot_ACCs columns, I don't have to specially add back in the space to match the original form.
        # While doing this iterating, start removing the accounted for 
        # identifiers from the second column string to end up with what is 
        # unaccounted for in the genenames column by the matching Uniprot_ACCs.
        # Importantly, only replace the first occurrence of the exact genename; 
        # this and the sorting I did above will help with related genes like 
        # `TBL1X` and `TBL1XR1`.
        parts = what_remains.split()
        if lookup_dict[original_ACC] in parts:
            parts.remove(lookup_dict[original_ACC]) #want the one that comes up
            # from the lookup_dict matching so that ones that have semi-colon in 
            # them removes both of the two name ones at the same time; the one 
            # I put in `new_genenames_list` above had the space removed and so 
            # if used that one the semi-colon related ones won't get deleted 
            # from what_remains properly and end up being specified erroneously 
            # as remaining
            what_remains = " ".join(parts)
        # need special handling for the cases where there is a semi-colon in the
        # matched genename and note want the one that comes up from the 
        # lookup_dict matching so that ones that have semi-colon in them removes 
        # both of the two name ones at the same time; the one. I put in 
        # `new_genenames_list` above had the space removed and so if used that 
        # one the semi-colon related ones won't get deleted from what_remains 
        # properly and end up being specified erroneously as remaining.
        '''
        if ';' in matched_genename and matched_genename.count(';') == 1: # some have more than one semi-colon and mostly I'll handle hardcoding below
            # Handle the semicolon case by finding and removing the exact pair
            parts = what_remains.split()
            i = 0
            while i < len(parts)-1:
                if parts[i].endswith(';'):
                    potential_pair = parts[i] + ' ' + parts[i+1]
                    if potential_pair == matched_genename:
                        # Remove both parts of the pair
                        parts.pop(i)
                        parts.pop(i)  # don't increment i since we removed two items
                        break
                    else:
                        i += 1
                else:
                    i += 1
            what_remains = ' '.join(parts)
        '''
        if ';' in matched_genename: # Handle gene names with semi-colon different for hu.MAP 2.0
            # Handle the semicolon case by finding and removing the first part of name
            parts = what_remains.split()
            first_part_matched_genename = matched_genename.split(';')[0]
            if first_part_matched_genename in parts:
                parts.remove(first_part_matched_genename)
                what_remains = ' '.join(parts)
        elif original_ACC in ['O15482','P23610','P62805','P68431','Q71DI3','P62807','P62807','P0C0S8','Q6ZMK1','P04745','P0DN76','A6NLF2','B9ZVM9','Q9Y4X1','Q12799','Q5VWM5','P0DN79','P0DN76']:
            # handle some special cases where there's semi-colons (plus a few additional cpecial cases) so default below won't work and so few that it isn't worth working out programmatically
            # SET UP TO Handle several special cases I noted, so proper text gets removed and don't add extra SPECIAL case to result CSV:
            if original_ACC == 'O15482':
                #matched_genename = 'TEX28; TEX28P1; TEX28P2' #for hu.MAP 3; ones with more than a single semi-colon. Just going to hardcode handling removal so not tagged as unaccounted for.
                matched_genename = 'TEX28'#for hu.MAP 2
            if original_ACC == 'P23610':
                #matched_genename = 'F8A1; F8A2; F8A3' #for hu.MAP 3
                matched_genename = 'F8A1' #for hu.MAP 2
            if original_ACC == 'P62805':
                #matched_genename = 'H4C1; H4C2; H4C3; H4C4; H4C5; H4C6; H4C8; H4C9; H4C11; H4C12; H4C13; H4C14; H4C15; H4C16' #for hu.MAP 3
                matched_genename = 'H4C1' #for hu.MAP 2
            if original_ACC == 'P68431':
                #matched_genename = 'H3C1; H3C2; H3C3; H3C4; H3C6; H3C7; H3C8; H3C10; H3C11; H3C12' #for hu.MAP 3
                matched_genename = 'H3C1' #for hu.MAP 2
            if original_ACC == 'Q71DI3':
                matched_genename = 'H3C15; H3C14; H3C13'
            if original_ACC == 'P62807':
                matched_genename = 'H2BC4; H2BC6; H2BC7; H2BC8; H2BC10'
            if original_ACC == 'P0C0S8':
                #matched_genename = 'H2AC11; H2AC13; H2AC15; H2AC16; H2AC17' #for hu.MAP 3
                matched_genename = 'H2AC11' #for hu.MAP 2
            if original_ACC == 'Q6ZMK1':
                matched_genename = 'CYHR1' # They use 'CYHR1' which doesn't exist in UniProt now, see https://www.uniprot.org/uniprotkb/Q6ZMK1/history ;'This entry has now been demerged. Its accession has been set as secondary accession in P0DTL5 and P0DTL6.'
            if original_ACC == 'P04745':
                matched_genename = 'AMY1A' # They use 'AMY1A' which doesn't exist in UniProt now, see https://www.uniprot.org/uniprotkb/P04745/history ;'This entry has now been demerged. Its accession has been set as secondary accession in P0DTE7, P0DTE8, and P0DUB6'
            if original_ACC == 'P0DN76':
                matched_genename = 'U2AF1L5' # They use 'U2AF1L5' which doesn't exist in UniProt now; like case for `CYHR1` and `AMY1A` above
            if original_ACC == 'A6NLF2':
                matched_genename = 'ELOA3D' # They use 'ELOA3D' which doesn't exist in UniProt now; like case for `CYHR1` and `AMY1A` above
            if original_ACC == 'B9ZVM9':
                matched_genename = 'TCP10L2' # They use 'TCP10L2' which doesn't exist in UniProt now; like case for `CYHR1` and `AMY1A` above
            if original_ACC == 'Q9Y4X1':
                matched_genename = 'UGT2A1' # They use 'UGT2A1' which doesn't exist in UniProt now; like case for `CYHR1` and `AMY1A` above
            if original_ACC == 'Q12799':
                matched_genename = 'TCP10' # They use 'TCP10' which doesn't exist in UniProt now; like case for `CYHR1` and `AMY1A` above
            if original_ACC == 'Q5VWM5':
                matched_genename = 'PRAMEF9' # They use 'PRAMEF9' which doesn't exist in UniProt now; like case for `CYHR1` and `AMY1A` above
            if original_ACC == 'P0DN79':
                matched_genename = 'CBSL' # They use 'CBSL' which doesn't exist in UniProt now; like case for `CYHR1` and `AMY1A` above
            # END SET UP FOR SEVERAL SPECIAL CASES WITH MANY SEMI-COLONS + other special cases where ACC is distinct
            what_remains = what_remains.replace(matched_genename,'')
        else:
            # Handle single genename case (when there is no semi-colon)
            parts = what_remains.split()
            # SET UP TO Handle several special cases I noted, so proper text gets removed and don't add extra SPECIAL case to resulting CSV:
            if matched_genename == 'SLC66A1LP':
                matched_genename = 'SLC66A1L' # makes it match what author provided file had for that one; they used one of its few synonyms
            if matched_genename == 'PRP4K':
                matched_genename = 'PRPF4B' # makes it match what author provided file had for that one; they used one of its few synonyms
            if matched_genename == 'CFAP263':
                matched_genename = 'CCDC113' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'CFAP337':
                matched_genename = 'WDR49' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'FERRY3':
                matched_genename = 'C12orf4' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'CFAP184':
                matched_genename = 'CCDC96' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'GARRE1':
                matched_genename = 'KIAA0355' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'SKIC8':
                matched_genename = 'WDR61' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'FHIP1B':
                matched_genename = 'FAM160A2' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'FHIP1A':
                matched_genename = 'FAM160A1' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'IFT70B':
                matched_genename = 'TTC30B' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'IFT70A':
                matched_genename = 'TTC30A' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'IFT56':
                matched_genename = 'TTC26' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'H2AC20':
                matched_genename = 'HIST2H2AC' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'STEEP1':
                matched_genename = 'CXorf56' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'ATP5MK':
                matched_genename = 'ATP5MD' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'MACIR':
                matched_genename = 'C5orf30' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'CYRIA':
                matched_genename = 'FAM49A' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'CYRIB':
                matched_genename = 'FAM49B' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'UQCC6':
                matched_genename = 'C12orf73' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'H2AC21':
                matched_genename = 'HIST2H2AB' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'POLR1H':
                matched_genename = 'ZNRD1' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'POLR1G':
                matched_genename = 'CD3EAP' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'POLR1F':
                matched_genename = 'TWISTNB' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'RUSF1':
                matched_genename = 'C16orf58' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'GARIN1A':
                matched_genename = 'FAM71F2' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'MARCHF5':
                matched_genename = 'MARCH5' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'H1-10':
                matched_genename = 'H1FX' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'H1-6':
                matched_genename = 'HIST1H1T' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'CEP20':
                matched_genename = 'FOPNL' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'ODAD3':
                matched_genename = 'CCDC151' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'SKIC2':
                matched_genename = 'SKIV2L' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'MIX23':
                matched_genename = 'CCDC58' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'H2BC17':
                matched_genename = 'HIST1H2BO' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'H2AX':
                matched_genename = 'H2AFX' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'H2BC5':
                matched_genename = 'HIST1H2BD' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'ARB2A':
                matched_genename = 'FAM172A' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'MTCL2':
                matched_genename = 'SOGA1' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'PSME3IP1':
                matched_genename = 'FAM192A' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'H2BC18':
                matched_genename = 'HIST2H2BF' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'YJU2B':
                matched_genename = 'CCDC130' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'CILK1':
                matched_genename = 'ICK' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'GFUS':
                matched_genename = 'TSTA3' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'ADGRE5':
                matched_genename = 'CD97' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'CERT1':
                matched_genename = 'CERT' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'ODAD2':
                matched_genename = 'ARMC4' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'ODAD4':
                matched_genename = 'TTC25' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'MTCL3':
                matched_genename = 'SOGA3' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'TRAPPC14':
                matched_genename = 'MAP11' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'SCAND3':
                matched_genename = 'ZBED9' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'CDIN1':
                matched_genename = 'C15orf41' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'CSTPP1':
                matched_genename = 'C11orf49' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'PHB1':
                matched_genename = 'PHB' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'ENTREP3':
                matched_genename = 'FAM189B' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'FAM200C':
                matched_genename = 'ZBED8' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'PABIR1':
                matched_genename = 'FAM122A' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'VCF1':
                matched_genename = 'FAM104A' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'PALS1':
                matched_genename = 'MPP5' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'DYNC2I2':
                matched_genename = 'WDR34' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'DYNLT2B':
                matched_genename = 'TCTEX1D2' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'DYNC2I1':
                matched_genename = 'WDR60' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'IFTAP':
                matched_genename = 'C11orf74' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'RARS1':
                matched_genename = 'RARS' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'LARS1':
                matched_genename = 'LARS' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'EPRS1':
                matched_genename = 'EPRS' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'EPRS1':
                matched_genename = 'QARS' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'MARS1':
                matched_genename = 'MARS' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'IARS1':
                matched_genename = 'IARS' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'DARS1':
                matched_genename = 'DARS' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'CIMIP4':
                matched_genename = 'TEX33' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'WARS1':
                matched_genename = 'WARS' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'H1-7':
                matched_genename = 'H1FNT' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'PABIR2':
                matched_genename = 'FAM122B' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'HYCC1':
                matched_genename = 'FAM126A' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'METTL13':
                matched_genename = 'EEF1AKNMT' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'CRIPTO':
                matched_genename = 'TDGF1' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'AIRIM':
                matched_genename = 'C1orf109' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'AFG2B':
                matched_genename = 'SPATA5L1' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'AFG2A':
                matched_genename = 'SPATA5' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'PALS2':
                matched_genename = 'MPP6' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'CARS1':
                matched_genename = 'CARS' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'TARS3':
                matched_genename = 'TARSL2' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'MIDEAS':
                matched_genename = 'ELMSAN1' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'ZNG1C':
                matched_genename = 'CBWD3' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'ARK2N':
                matched_genename = 'C18orf25' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'ZFHX3':
                matched_genename = 'C16orf47' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'CEP43':
                matched_genename = 'FGFR1OP' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'VARS1':
                matched_genename = 'VARS' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'BBLN':
                matched_genename = 'C9orf16' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'DNAI3':
                matched_genename = 'WDR63' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'ATOSA':
                matched_genename = 'FAM214A' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'CIMAP1D':
                matched_genename = 'ODF3L2' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'BLTP2':
                matched_genename = 'KIAA0100' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'SARS1':
                matched_genename = 'SARS' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'PGAP6':
                matched_genename = 'TMEM8A' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'SLX9':
                matched_genename = 'FAM207A' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'CBY2':
                matched_genename = 'SPERT' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'BLTP3A':
                matched_genename = 'UHRF1BP1' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'TASL':
                matched_genename = 'CXorf21' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'UQCC4':
                matched_genename = 'CCSMST1' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'DNAAF10':
                matched_genename = 'WDR92' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'GBA1':
                matched_genename = 'GBA' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'MARCHF9':
                matched_genename = 'MARCH9' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'TAFAZZIN':
                matched_genename = 'TAZ' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'KICS2':
                matched_genename = 'C12orf66' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'KGD4':
                matched_genename = 'MRPS36' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'FHIP2A':
                matched_genename = 'FAM160B1' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'SAXO5':
                matched_genename = 'TEX45' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'SPMIP6':
                matched_genename = 'SMRP1' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'AARS1':
                matched_genename = 'AARS' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'ENTREP1':
                matched_genename = 'FAM189A2' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'BMAL1':
                matched_genename = 'ARNTL' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'GET3':
                matched_genename = 'ASNA1' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'TMT1A':
                matched_genename = 'METTL7A' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'YARS1':
                matched_genename = 'YARS' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'MACROH2A2':
                matched_genename = 'H2AFY2' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'PHAF1':
                matched_genename = 'C16orf70' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'CIBAR1':
                matched_genename = 'FAM92A' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'DNAAF11':
                matched_genename = 'LRRC6' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'SANBR':
                matched_genename = 'KIAA1841' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'GET1':
                matched_genename = 'WRB' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'TMT1B':
                matched_genename = 'METTL7B' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'MTARC2':
                matched_genename = 'MARC2' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'SPRING1':
                matched_genename = 'C12orf49' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'INTS15':
                matched_genename = 'C7orf26' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'DNAAF8':
                matched_genename = 'C16orf71' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'NTAQ1':
                matched_genename = 'WDYHV1' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'UTP25':
                matched_genename = 'DIEXF' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'BMAL2':
                matched_genename = 'ARNTL2' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'QNG1':
                matched_genename = 'C9orf64' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'NHSL3':
                matched_genename = 'KIAA1522' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'SEPTIN4':
                matched_genename = 'C17orf47' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'SPMIP4':
                matched_genename = 'C7orf31' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'PTGR3':
                matched_genename = 'ZADH2' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'EEIG2':
                matched_genename = 'FAM102B' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'IFT25':
                matched_genename = 'HSPB11' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'NOPCHAP1':
                matched_genename = 'C12orf45' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'ZNG1A':
                matched_genename = 'CBWD1' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'ZNG1B':
                matched_genename = 'CBWD2' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'ZNG1F':
                matched_genename = 'CBWD6' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'FHIP2B':
                matched_genename = 'FAM160B2' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'MARCHF6':
                matched_genename = 'MARCH6' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'ELAPOR2':
                matched_genename = 'KIAA1324L' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'GOLM2':
                matched_genename = 'CASC4' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'STING1':
                matched_genename = 'TMEM173' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'NHERF2':
                matched_genename = 'SLC9A3R2' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'SKIC3':
                matched_genename = 'TTC37' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'TMEM183BP':
                matched_genename = 'TMEM183B' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'BPNT2':
                matched_genename = 'IMPAD1' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'MYL11':
                matched_genename = 'MYLPF' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'MACROH2A1':
                matched_genename = 'H2AFY' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'NHERF1':
                matched_genename = 'SLC9A3R1' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'BLTP3B':
                matched_genename = 'UHRF1BP1L' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'ADISSP':
                matched_genename = 'C20orf27' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'METTL25B':
                matched_genename = 'RRNAD1' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'H3-5':
                matched_genename = 'H3F3C' # makes it match what author provided file had for that one; they used its synonym
            if matched_genename == 'CENATAC':
                matched_genename = 'CCDC84' # makes it match what author provided file had for that one; they used what UniProt has under 'ORF names'
            if matched_genename == 'MTNAP1':
                matched_genename = 'C17orf80' # makes it match what author provided file had for that one; they used what UniProt has under 'ORF names'
            if matched_genename == 'CEP15':
                matched_genename = 'C3orf14' # makes it match what author provided file had for that one; they used what UniProt has under 'ORF names'
            if matched_genename == 'PALM2AKAP2':
                matched_genename = 'AKAP2' # makes it match what author provided file had for that one; they used an abbreviation of what UniProt has under 'Alternative names'
            # END SET UP FOR SEVERAL SPECIAL CASES
            if matched_genename in parts:
                parts.remove(matched_genename)
                what_remains = ' '.join(parts)

    # now decide if anything extra has to be added to new_genenames_string by
    # accounting for everything in original and adding what remains. Anything 
    # that gets added will need a 'SPECIAL_'-branded id placeholder added to the
    # new_Uniprot_ACCs_list so that things remain balanced
    if what_remains.strip():
        genenames_had_unaccounted_for = True # set this for accounting
        what_genenames_had_unaccounted_for = what_remains.strip() # set this for accounting
        # now iterate on what splitting that string at the spaces returns and 
        # do special handling or if not in what needs special handling then
        # add a corresponding 'SPECIAL_'-labeled entry placeholder 
        unaccounted_genenames_list = what_genenames_had_unaccounted_for.split()
        for unaccounted_genename in unaccounted_genenames_list:
            if unaccounted_genename in special_genename_lookup_dict:
                new_Uniprot_ACCs_list.append(special_genename_lookup_dict[unaccounted_genename]) 
                new_genenames_list.append(unaccounted_genename)
            else:
                new_Uniprot_ACCs_list.append('SPECIAL_unaccounted_gene') # NOTE, I used iterating on running this function & finding this occurence in the data to progressively add handling two types of 'special cases' (one type being those with threee or more semi-colons and the other being where the author-provided file use a synonyms) and ultimately end up with NO instances of `SPECIAL_unaccounted_gene` being necessary in my fixed in-order matched, balanced verison of the data.
                new_genenames_list.append(unaccounted_genename)
    assert (len(new_Uniprot_ACCs_list) == len(new_genenames_list)) #sanity 
    # check; they definitely should balance still
    new_Uniprot_ACCs_string = " ".join(new_Uniprot_ACCs_list)
    new_genenames_string = " ".join(new_genenames_list)
    # Now that done making in-order matched & balanced content, assign the 
    # appropriate text for those two columns to be returned when row returned at 
    # end of this entire function (before getting to end of this function 
    # though, will do some accounting)
    row['Uniprot_ACCs'] = new_Uniprot_ACCs_string
    row['genenames'] = new_genenames_string

    # Check original state for collecting some details
    '''
    for idx,oa in enumerate(original_ACCs_as_list):
        if original_genenames_as_list[idx] != lookup_dict[oa]:
            matching_originally = False
            break

    matching_originally = all(
        original_genenames_as_list[i] == lookup_dict[oa]
        for i, oa in enumerate(original_ACCs_as_list)
    )
    '''
    if len(original_ACCs_as_list) == len(original_genenames_as_list):
        balanced_originally = True
        # Only worth checking matching if balanced because otherwise get out `IndexError: list index out of range` as try to check `original_genenames_as_list[i]`
        matching_originally = all(
            original_genenames_as_list[i] == lookup_dict[oa]
            for i, oa in enumerate(original_ACCs_as_list)
        )
    if ';' in row['genenames']:
        involves_semicolon = True
    # Plus check the new state as part of the information being collected; the rows in the tracker will correspond to the fixed CSV to be made and so it will be pertinent to the corresponding row and I don't want to pollute the one to be fixed
    if any('SPECIAL' in item for item in new_Uniprot_ACCs_list + new_genenames_list):
        SPECIAL_involved =True
    # want to track, 'matching_originally','balanced originally','involves_semicolon','SPECIAL_involved','genenames_had_unaccounted_for','what_genenames_had_unaccounted_for'
    # This way later can use count of the numbers there to easy to see what is going on.
    tracker.tracking_list.append(
                            {
                            'matching_originally':matching_originally, 
                            'balanced_originally':balanced_originally, 
                            'involves_semicolon': involves_semicolon,
                            'SPECIAL_involved': SPECIAL_involved,
                            'genenames_had_unaccounted_for': genenames_had_unaccounted_for,
                            'what_genenames_had_unaccounted_for': what_genenames_had_unaccounted_for
                            })  # Append to the class attribute list a dictionary
    return row

tracker = Tracker()
fixed_df = fixed_df.apply(make_ordered_fix_and_collect_info_about_balance, args=(tracker,), axis=1)
# Save this as CSV
fixed_df.to_csv("humap2_complexes_20200809InOrderMatched.csv",index = False)
# will add to the tracker df as columns the lists corresponding to the values of the list of dictionaries
#matching_originally_values = [d['matching_originally'] for d in tracker]
#balanced_originally_values = [d['balanced_originally'] for d in tracker]
#involves_semicolon_values = [d['involves_semicolon'] for d in tracker]
#SPECIAL_involved_values = [d['SPECIAL_involved'] for d in tracker]
#genenames_had_unaccounted_for_values = [d['genenames_had_unaccounted_for'] for d in tracker]
#what_genenames_had_unaccounted_for_values = [d['what_genenames_had_unaccounted_for'] for d in tracker]
#tracker_results_df['matching_originally'] = matching_originally_values
tracker_results_df['matching_originally'] = [d['matching_originally'] for d in tracker.tracking_list] 
tracker_results_df['balanced_originally'] = [d['balanced_originally'] for d in tracker.tracking_list] 
tracker_results_df['involves_semicolon'] = [d['involves_semicolon'] for d in tracker.tracking_list]
tracker_results_df['SPECIAL_involved'] = [d['SPECIAL_involved'] for d in tracker.tracking_list]
tracker_results_df['genenames_had_unaccounted_for'] = [d['genenames_had_unaccounted_for'] for d in tracker.tracking_list]
tracker_results_df['what_genenames_had_unaccounted_for'] = [d['what_genenames_had_unaccounted_for'] for d in tracker.tracking_list]

In [21]:
tracker_results_df.head()

,HuMAP2_ID,Confidence,Uniprot_ACCs,genenames,matching_originally,balanced_originally,involves_semicolon,SPECIAL_involved,genenames_had_unaccounted_for,what_genenames_had_unaccounted_for
0,HuMAP2_00000,3,Q9BQS8 O95900,FYCO1 TRUB2,False,True,False,False,False,NOTHING
1,HuMAP2_00001,4,P08133 Q15797 Q99426 Q9H4M9 P68402 Q15102,ANXA6 SMAD1 TBCB EHD1 PAFAH1B2 PAFAH1B3,False,True,False,False,False,NOTHING
2,HuMAP2_00002,5,Q93062 Q9NZC3 Q9UF11 Q15038 Q6ZRY4 A1KXE4 O432...,RBPMS GDE1 PLEKHB1 DAZAP2 RBPMS2 FAM168B RBFOX...,False,True,False,False,False,NOTHING
3,HuMAP2_00003,5,Q15836 Q16563 Q29983 Q8WUM9 O14974 Q9Y5Y0 Q149...,VAMP3 SYPL1 MICA SLC20A1 PPP1R12A FLVCR1 DRAP1...,False,True,False,False,False,NOTHING
4,HuMAP2_00004,4,Q8WV99 Q9NQT8 Q9H672 P20774 Q49A92,ZFAND2B KIF13B ASB7 OGN C8orf34,False,True,False,False,False,NOTHING


**Initially running the above two steps with the code I worked out with the hu.MAP 3.0 data revealed some additional special cases lurking in the hu.MAP 2.0 data that I had to deal with:**
(See the special handling code in the cell that two cells up for a full list; you'll need to compare that to the special handling section in  [the notebook 'Jupyter Notebook Standardizing Identifier Order In Adjacent Columns in hu.MAP2-provided CSV'](https://nbviewer.org/github/fomightez/humap3-binder/blob/main/additional_nbs/standardizing_initial_data/Standardizing_identifier_order_in_humap3-provided_csv.ipynb) to fully see the many of them but they fall into two categories so far: **more synonym names used** and **cases where genename is used for what is no longer in UniProt**.)  
For dealing with the synonym names I had to add some special handling beyond that I had for the hu.MAP 3.0 data. To do that, I donwloaded an intermediate draft result, `DRAFThumap2_complexes_20200809InOrderMatched.csv`, that still had a lot (863 according to 'Find') of cases of `SPECIAL_unaccounted_gene` and I ran the following code to generate many more entries in the last section of special handling for accounting for what is missing in Uniprot_ACCs column, corresponding to `# makes it match what author provided file had for that one; they used its synonym`:
The authors also had some like, `CYHR1` and `AMY1A`, that I found when developing automating making code dealing with synonyms handling. (These ones turned out to be revealed when coming across those without `'genes' in uniprot_record`.) A good percentage of these had been removed from UniProt and so no gene name even though the authors used one, and so I added special handling for the UniProt accessions for those, too. Plus, some of these involving gaps in the `Uniprot_ACCs` column seemed to just be where something got deleted as a large percent where pseudogenes but the genename remained.
I added handling to the cell above to deal with many of these now, and so..... hopefully

...eliminated all those, so at this point `genenames_had_unaccounted_for` and	`what_genenames_had_unaccounted_for` are moot and can be removed from the tracker data.  
Let's verify that here and then remove those two tracking columns. ??????

In [22]:
unf_count_df = tracker_results_df.copy().value_counts('genenames_had_unaccounted_for').reset_index()
unf_count_df.head()

,genenames_had_unaccounted_for,count
0,False,6736
1,True,229


In [23]:
tracker_results_df = tracker_results_df.drop(columns=['genenames_had_unaccounted_for', 'what_genenames_had_unaccounted_for'])
tracker_results_df.head()

,HuMAP2_ID,Confidence,Uniprot_ACCs,genenames,matching_originally,balanced_originally,involves_semicolon,SPECIAL_involved
0,HuMAP2_00000,3,Q9BQS8 O95900,FYCO1 TRUB2,False,True,False,False
1,HuMAP2_00001,4,P08133 Q15797 Q99426 Q9H4M9 P68402 Q15102,ANXA6 SMAD1 TBCB EHD1 PAFAH1B2 PAFAH1B3,False,True,False,False
2,HuMAP2_00002,5,Q93062 Q9NZC3 Q9UF11 Q15038 Q6ZRY4 A1KXE4 O432...,RBPMS GDE1 PLEKHB1 DAZAP2 RBPMS2 FAM168B RBFOX...,False,True,False,False
3,HuMAP2_00003,5,Q15836 Q16563 Q29983 Q8WUM9 O14974 Q9Y5Y0 Q149...,VAMP3 SYPL1 MICA SLC20A1 PPP1R12A FLVCR1 DRAP1...,False,True,False,False
4,HuMAP2_00004,4,Q8WV99 Q9NQT8 Q9H672 P20774 Q49A92,ZFAND2B KIF13B ASB7 OGN C8orf34,False,True,False,False


In [ ]:
NEXT add in:
- checking for duplicates by using `%run .ipynb` to run the notebook I made.

Now use the 'tracker_results_df' that parallels the dataframe I used to make the resulting CSV to get some idea of numbers of what rows were originally in-order matchings & balanced, plus some details of the results I made to fix things to be closer to what will easiluy make tidy data using Pandas `explode()`:

In [17]:
mo_count_df = tracker_results_df.copy().value_counts('matching_originally').reset_index()
mo_count_df.head()

,matching_originally,count
0,False,5651
1,True,1314


In [18]:
mo_count_df = tracker_results_df.copy().value_counts('matching_originally', normalize=True).reset_index()
mo_count_df.head()

,matching_originally,proportion
0,False,0.811342
1,True,0.188658


So not even 20% were matching the order in the two columns, `Uniprot_ACCs` & `genenames`, in the original CSV. 

In [19]:
bal_count_df = tracker_results_df.copy().value_counts('balanced_originally').reset_index()
bal_count_df.head()

,balanced_originally,count
0,True,6731
1,False,234


I had this number earlier because it was what I checked initially because it was an easy check to gauge how tidy the author-provided hu.MAP 2.0 data might be.

In [24]:
sc_count_df = tracker_results_df.copy().value_counts('involves_semicolon').reset_index()
sc_count_df.head()

,involves_semicolon,count
0,False,6879
1,True,86


I don't think original had semi-colon and so ones here should be well-handled. So not as big a deal now that standardized. Just printing it now since I did same for fixed hu.MAP 3.0 data for just being aware.


In my 'fixed' version of the CSV data what is the amount of 'SPECIAL' cases I identified?


In [25]:
sp_count_df = tracker_results_df.copy().value_counts('SPECIAL_involved').reset_index()
sp_count_df.head()

,SPECIAL_involved,count
0,False,6901
1,True,64


In [26]:
sp_count_df = tracker_results_df.copy().value_counts('SPECIAL_involved', normalize=True).reset_index()
sp_count_df.head()

,SPECIAL_involved,proportion
0,False,0.990811
1,True,0.009189


So there's quite a small fraction, below above 1%, that are 'special' cases in there now.  
And even if these are special there is matching information in both colunns in my 'fixed' CSV to keep things explodable and utimately tidy.

**Use ` humap2_complexes_20200809InOrderMatched.csv` saved above the 'accounting' I just did.**

------

In [ ]:
# for development; to keep kernel active
import time

def executeSomething():
    #code here
    print ('.')
    time.sleep(480) #60 seconds times 8 minutes

while True:
    executeSomething()

.
